In [12]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [13]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/db_congress.db"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Members_Metadata = Base.classes.members



C:\Users\porphan\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [69]:
def line(state):
    stmt = db.session.query(Members_Metadata).statement
    df = pd.read_sql_query(stmt, db.session.bind)
    
    # Filter the data for the state selected
    df_filtered = df.loc[df['State'] == state].reset_index()
    
    df_grouped = df['SwornInAge'].groupby([df['State'], df['Year']])
    df_all = df['SwornInAge'].groupby(df['Year'])

    # Format the data to send as json
#     data = {
#         "Year": df.Year.values.tolist(),
#         "SwornInAge": df.SwornInAge.values.tolist(),
#         "State": df.State.values.tolist(),
#     }
#     print(state)
#     print(df_all.mean())
#     print(df_grouped.mean())
    return df_filtered
#     return jsonify(data)
line('MI')

,index,UserId,CongressID,Chamber,FirstName,LastName,DateOfBirth,Gender,Party,State,Year,SwornInAge
0,0,1,104,Senate,Spencer,Abraham,1952-06-12,M,R,MI,1995,42.56
1,67,68,104,Senate,Carl,Levin,1934-06-28,M,D,MI,1995,60.52
2,104,105,105,Senate,Spencer,Abraham,1952-06-12,M,R,MI,1997,44.56
3,169,170,105,Senate,Carl,Levin,1934-06-28,M,D,MI,1997,62.52
4,204,205,106,Senate,Spencer,Abraham,1952-06-12,M,R,MI,1999,46.56
5,268,269,106,Senate,Carl,Levin,1934-06-28,M,D,MI,1999,64.52
6,372,373,107,Senate,Carl,Levin,1934-06-28,M,D,MI,2001,66.52
7,400,401,107,Senate,Debbie,Stabenow,1950-04-29,F,D,MI,2001,50.68
8,476,477,108,Senate,Carl,Levin,1934-06-28,M,D,MI,2003,68.52
9,503,504,108,Senate,Debbie,Stabenow,1950-04-29,F,D,MI,2003,52.68


In [50]:
line('MO')

,UserId,CongressID,Chamber,FirstName,LastName,DateOfBirth,Gender,Party,State,Year,SwornInAge
0,1,104,Senate,Spencer,Abraham,1952-06-12,M,R,MI,1995,42.56
1,2,104,Senate,Daniel,Akaka,1924-09-11,M,D,HI,1995,70.31
2,3,104,Senate,John,Ashcroft,1942-05-09,M,R,MO,1995,52.65
3,4,104,Senate,Max,Baucus,1941-12-11,M,D,MT,1995,53.06
4,5,104,Senate,Robert,Bennett,1933-09-18,M,R,UT,1995,61.29
5,6,104,Senate,Joseph,Biden,1942-11-20,M,D,DE,1995,52.12
6,7,104,Senate,Jeff,Bingaman,1943-10-03,M,D,NM,1995,51.25
7,8,104,Senate,Christopher,Bond,1939-03-06,M,R,MO,1995,55.83
8,9,104,Senate,Barbara,Boxer,1940-11-11,F,D,CA,1995,54.14
9,10,104,Senate,William,Bradley,1943-07-28,M,D,NJ,1995,51.44
